In [7]:
import jsonlines
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
FILE_TEST_PRED = "[TEST]_pred_weights.jsonl"
FILE_TEST_PRED_ONLY_BERT = "pred_test_with_bert.jsonl"
FILE_TEST_TRUE = "scicite/data/acl-arc/test.jsonl"

FILE_DEV_PRED = "[DEV]_pred_weights.jsonl"
FILE_DEV_PRED_ONLY_BERT = "pred_dev_with_bert.jsonl"
FILE_DEV_TRUE = "scicite/data/acl-arc/dev.jsonl"

FILE_TRAIN_PRED = "[TRAIN]_pred_weights.jsonl"
FILE_TRAIN_PRED_ONLY_BERT = "pred_train_with_bert.jsonl"
FILE_TRAIN_TRUE = "scicite/data/acl-arc/train.jsonl"

CLASSES = ['Background', 'Uses', 'CompareOrContrast', 'Motivation', 'Extends', 'Future']

In [ ]:
def compute_all_patterns_features(item: Dict):
    formulaic_features, fn_1, __ = get_formulaic_features(item['sents_before'][0], prefix='InCitSent:')
    agent_features, fn_2, _ = get_agent_features(item['sents_before'][0], prefix='InCitSent:')
    fn_3, fn_4 = [], []
    
    formulaic_clause_features = formulaic_features
    agent_clause_features = agent_features
    if len(item['sents_before']) > 1:
        for cur_sentence in item['sents_before'][1:]:
            _formulaic_features, fn_3, _ = get_formulaic_features(cur_sentence, prefix='InClause:')
            _agent_features, fn_4, _ = get_agent_features(cur_sentence, prefix='InClause:')
            formulaic_clause_features = [f_1 or f_2 for f_1, f_2 in zip(formulaic_clause_features,
                                                                        _formulaic_features)]
            agent_clause_features = [f_1 or f_2 for f_1, f_2 in zip(agent_clause_features,
                                                                    _agent_features)]
    for cur_sentence in item['sents_after']:
        _formulaic_features, _, _ = get_formulaic_features(cur_sentence, prefix='InClause:')
        _agent_features, _, _ = get_agent_features(cur_sentence, prefix='InClause:')
        formulaic_clause_features = [f_1 or f_2 for f_1, f_2 in zip(formulaic_clause_features,
                                                                    _formulaic_features)]
        agent_clause_features = [f_1 or f_2 for f_1, f_2 in zip(agent_clause_features,
                                                                _agent_features)]
        

    x = formulaic_features + agent_features + formulaic_clause_features + agent_clause_features 
    y = CLASSES.index(item['intent'])
    feature_names = fn_1 + fn_2
    if fn_3:
        feature_names += fn_3 + fn_4 
    return x, y, feature_names

In [ ]:
def get_true_examples_patterns(true_file, pred_file)
    result = {cls: 0 for cls in CLASSES}
    
    # true values
    true = dict()
    with jsonlines.open(true_file) as reader:
        for obj in reader:
            true[obj['citation_id']] = obj
    
    # pred values
    pairs = dict()
    with jsonlines.open(pred_file) as reader:
        for obj in reader:
            if true[obj['citation_id']]['intent'] != obj['prediction']:
                new_obj = true[obj['citation_id']]
                new_obj['prediction'] = obj['prediction']
                result.append(new_obj)
    
    return result

In [11]:
def get_errors(true_file, pred_file):
    result = []
    
    # true values
    true = dict()
    with jsonlines.open(true_file) as reader:
        for obj in reader:
            true[obj['citation_id']] = obj
    
    # pred values
    pairs = dict()
    with jsonlines.open(pred_file) as reader:
        for obj in reader:
            if true[obj['citation_id']]['intent'] != obj['prediction']:
                new_obj = true[obj['citation_id']]
                new_obj['prediction'] = obj['prediction']
                result.append(new_obj)
    
    return result

In [12]:
errors = get_errors(FILE_TEST_TRUE, FILE_TEST_PRED)

In [14]:
for _ in errors:
    print(f"True={_['intent']} Predict={_['prediction']}")

True=Uses Predict=Future
True=Motivation Predict=Uses
True=Background Predict=CompareOrContrast
True=Uses Predict=Motivation
True=CompareOrContrast Predict=Background
True=CompareOrContrast Predict=Extends
True=Uses Predict=Background
True=CompareOrContrast Predict=Background
True=Uses Predict=Extends
True=CompareOrContrast Predict=Motivation
True=Uses Predict=Background
True=CompareOrContrast Predict=Background
True=Uses Predict=Background
True=Background Predict=Future
True=Background Predict=Uses
True=Motivation Predict=Uses
True=Motivation Predict=Background
True=Background Predict=Uses
True=CompareOrContrast Predict=Background
True=CompareOrContrast Predict=Background
True=Background Predict=Uses
True=Background Predict=CompareOrContrast
True=Motivation Predict=Background
True=Background Predict=Uses
True=Extends Predict=Background
True=Background Predict=CompareOrContrast
True=Motivation Predict=CompareOrContrast
